# Plot x for fixed y & z

In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
TIME_LIM = 30  # seconds or time steps

In [4]:
def varyX(minX, maxX, y, z, timeLimit=30):
    xs = range(minX, maxX+1)
    probs = {}
    
    for x in xs:
        state = (x,y,z)
        # Set up signal handler for SIGALRM, saving previous value
        oldHandler = signal.signal(signal.SIGALRM, sigalarmHandler)
        # Start timer
        signal.alarm(TIME_LIM)
        try:
            probs[x] = LoserAnalysis(state).getHittingProb()
        except TimeoutException:
            print(f'Took too long to find the hitting prob for state {state}')
        except RecursionError:
            print(f'maximum recursion depth exceeded for state {state}')
        finally:
            # Turn off timer
            signal.alarm(0)
            # Restore handler to previous value
            signal.signal(signal.SIGALRM, oldHandler)
        
    # use numerical approximation for the unrecorded states
    missedStates = [(x,y,z) for x in xs if x not in probs]
    approxProbs = {s[0]:LoserAnalysis(s).getHittingProb(approx=True) for s in missedStates}
    
    return probs, approxProbs

In [5]:
def varyX_memoised(minX, maxX, y, z, t=10):
    return {}, {x:Q(x,y,z,t) for x in range(minX, maxX+1)} # no exact probs - just to comply with the return values of varyX

In [6]:
# Automate the process
def plotXforFixedYZ(minX, maxX, y, z, filename, probFunction=varyX, t=30, markerSize=2, color='r'):
    """
    Plots x values within the range [`minX`, `maxX`] (inclusive) for fixed y and z.
    The `timeLimit` (in seconds) casts a limit on the computation time of the hitting prob for each state.
    Saves the plot to a file provided by the string `figName`.
    """
    exactProbs, approxProbs = probFunction(minX, maxX, y, z, t)
    
    # plot
    plt.scatter([x for x in approxProbs.keys()], approxProbs.values(), 
                color=color, label='approx probs', s=markerSize)
    
    if probFunction == varyX: # has a mixture of exact and approx probs
        plt.scatter([x for x in exactProbs.keys()], exactProbs.values(), 
                color='b', label='exact probs', s=markerSize)
        plt.legend()

    # add graph features
    plt.xlabel('x')
    plt.ylabel('P(loser = Player 1)')
    plt.title(f"Hitting probabilities for different x values (y = {y}, z = {z})")

    plt.savefig(filename)
    plt.show()

In [4]:
class TimeoutException(Exception):
    """
    Custom exception for timeout.
    """
    pass

def sigalarmHandler(signum, frame):
    """
    Handler function to be called when SIGALRM is received.
    """
    # We get signal!
    raise TimeoutException()

# Fixed sum, varying x, y, z

In [7]:
import matplotlib as mpl
import matplotlib.cm as cm

In [9]:
def mapProbsToColors(ps):
    minima = min(ps)
    maxima = max(ps)

    norm = mpl.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap=cm.gist_rainbow)

    colors = [mapper.to_rgba(p)[:3] for p in ps]  # only keep the rgb (but not a) values
    return colors

In [13]:
def plotFixedSum(N, filename, t=30, colorMap=cm.gist_rainbow):
    precomp(N, t)
    
    states = [(N-y-z, y, z) for y in range(N+1) for z in range(N+1) if y+z <= N]
    ys = [s[1] for s in states]
    zs = [s[2] for s in states]
    ps = [Q(x, y, z, 10) for x,y,z in states]  # losing probs
    
    colors = mapProbsToColors(ps)
    
    plt.scatter(ys, zs, color=colors)
    plt.title(f'Hitting probs for x + y + z = {N}')
    plt.xlabel('y')
    plt.ylabel('z')
    plt.savefig(filename)
    
    plt.show()